<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/headFNs_expms_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup

In [1]:
%%capture
%pip install git+https://github.com/redwoodresearch/Easy-Transformer.git
%pip install einops datasets transformers fancy_einsum

In [2]:
from copy import deepcopy
import torch

assert torch.cuda.device_count() == 1
from tqdm import tqdm
import pandas as pd
import torch
import torch as t
from easy_transformer.EasyTransformer import (
    EasyTransformer,
)
from time import ctime
from functools import partial

import numpy as np
from tqdm import tqdm
import pandas as pd

from easy_transformer.experiments import (
    ExperimentMetric,
    AblationConfig,
    EasyAblation,
    EasyPatching,
    PatchingConfig,
)
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import random
import einops
from IPython import get_ipython
from copy import deepcopy
from easy_transformer.ioi_dataset import (
    IOIDataset,
)
from easy_transformer.ioi_utils import (
    path_patching,
    max_2d,
    CLASS_COLORS,
    show_pp,
    show_attention_patterns,
    scatter_attention_and_contribution,
)
from random import randint as ri
from easy_transformer.ioi_circuit_extraction import (
    do_circuit_extraction,
    get_heads_circuit,
    CIRCUIT,
)
from easy_transformer.ioi_utils import logit_diff, probs
from easy_transformer.ioi_utils import get_top_tokens_and_probs as g

ipython = get_ipython()
if ipython is not None:
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

 Initialise model (use larger N or fewer templates for no warnings about in-template ablation)

In [3]:
model = EasyTransformer.from_pretrained("gpt2").cuda()
# model = EasyTransformer.from_pretrained("gpt2")
model.set_use_attn_result(True)

/usr/local/lib/python3.10/dist-packages/easy_transformer/components.py:616: UserWarning: Moved LN1 to the attention block
  warnings.warn("Moved LN1 to the attention block")


Moving model to device:  cuda
Finished loading pretrained model gpt2 into EasyTransformer!


# Generate dataset with multiple prompts

In [4]:
def generate_prompts_list():
    prompts_list = []
    for i in range(1, 98):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list()

In [5]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if key != 'text']:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

In [6]:
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

# Copy score

In [7]:
def get_copy_scores(model, layer, head, dataset, verbose=False, neg=False, print_tokens=True):
    cache = {}
    model.cache_some(cache, lambda x: x == "blocks.0.hook_resid_post")
    model(dataset.toks.long())
    if neg:
        sign = -1
    else:
        sign = 1
    z_0 = model.blocks[1].attn.ln1(cache["blocks.0.hook_resid_post"])

    v = torch.einsum("eab,bc->eac", z_0, model.blocks[layer].attn.W_V[head])
    v += model.blocks[layer].attn.b_V[head].unsqueeze(0).unsqueeze(0)

    o = sign * torch.einsum("sph,hd->spd", v, model.blocks[layer].attn.W_O[head])
    logits = model.unembed(model.ln_final(o))

    k = 5
    n_right = 0

    pred_tokens_dict = {}
    words_moved = []
    # get the keys from the first prompt in the dataset
    words = [key for key in dataset.prompts[0].keys() if key != 'text']

    for seq_idx, prompt in enumerate(dataset.prompts):
        for word in words:
            pred_tokens = [
                model.tokenizer.decode(token)
                for token in torch.topk(
                    logits[seq_idx, dataset.word_idx[word][seq_idx]], k
                ).indices
            ]
            pred_tokens_dict[prompt[word]] = pred_tokens
            if " " + prompt[word] in pred_tokens:
                n_right += 1
                words_moved.append(prompt[word])

    percent_right = (n_right / (dataset.N * len(words))) * 100
    print(f"Copy circuit for head {layer}.{head} (sign={sign}) : Top {k} accuracy: {percent_right}%")

    if print_tokens == True:
        return pred_tokens_dict
    else:
        return words_moved

## Get important heads

Find what heads are specific to certain inputs, and what's common to the template.

Get important heads from: circuit_expms_template.ipynb (Section: print top heads. Copy output of 'top_indices'; put on one line using chatgpt)

NOTE: not all attention heads just copy, so use attention patterns to determine which ones copy to refine this list of heads

(Eg. if you copy all the top heads from IOI, only 9.9 and 10.0 are name movers while other heads are "S-inhibition", "induction", "duplicate", so only the name movers + backup NM will have top accuracy)

In [8]:
top_val = [(0, 10), (0, 1), (5,5), (6,1), (7, 10), (8,8), (7,11), (8,11), (9,1), (9,5), (10,7)]
for index, (layer, head) in enumerate(top_val):
    print(index, get_copy_scores(model, layer, head, dataset, print_tokens=False))

Copy circuit for head 0.10 (sign=1) : Top 5 accuracy: 0.0%
0 []
Copy circuit for head 0.1 (sign=1) : Top 5 accuracy: 0.0%
1 []
Copy circuit for head 5.5 (sign=1) : Top 5 accuracy: 0.25773195876288657%
2 ['12']
Copy circuit for head 6.1 (sign=1) : Top 5 accuracy: 1.2886597938144329%
3 ['5', '11', '18', '18', '18']
Copy circuit for head 7.10 (sign=1) : Top 5 accuracy: 75.25773195876289%
4 ['2', '3', '4', '3', '4', '5', '4', '5', '6', '5', '6', '7', '6', '7', '8', '7', '8', '9', '8', '9', '10', '9', '10', '11', '10', '11', '12', '11', '12', '13', '12', '13', '14', '13', '14', '15', '14', '15', '16', '15', '16', '17', '16', '17', '18', '17', '18', '19', '18', '19', '20', '19', '20', '21', '20', '21', '22', '21', '22', '23', '22', '23', '24', '23', '24', '25', '24', '25', '26', '25', '26', '27', '26', '27', '28', '27', '28', '29', '28', '29', '30', '29', '30', '31', '30', '31', '32', '31', '32', '33', '32', '33', '34', '33', '34', '35', '34', '35', '36', '35', '36', '37', '36', '37', '38', 

Look at heads with strong pos on heatmap

In [9]:
get_copy_scores(model, 9, 1, dataset)

Copy circuit for head 9.1 (sign=1) : Top 5 accuracy: 59.27835051546392%


{'1': [' one', ' needed', ' preferred', ' single', '2'],
 '2': [' third', ' fourth', 'third', '3', ' fifth'],
 '3': [' fourth', ' third', ' fifth', 'Fourth', 'fourth'],
 '4': [' fifth', ' sixth', ' seventh', 'fifth', 'five'],
 '5': [' sixth', ' seventh', ' fifth', '6', ' eighth'],
 '6': [' seventh', ' Seventh', ' sixth', '7', ' eighth'],
 '7': [' seventh', ' eighth', ' ninth', ' VIII', ' Seventh'],
 '8': [' ninth', ' eighth', 'ighth', ' seventh', '9'],
 '9': [' ninth', ' seventh', ' tenth', ' eighth', ' sixth'],
 '10': [' tenth', ' seventh', ' eighth', ' ninth', ' sixth'],
 '11': [' eighth', ' seventh', ' specific', ' 12', '12'],
 '12': [' seventh', ' eighth', '13', ' specific', '14'],
 '13': [' seventh', '14', ' 14', ' eighth', 'Only'],
 '14': [' eighth', ' seventh', ' fifth', ' maximum', ' ninth'],
 '15': [' seventh', ' eighth', ' sixth', ' fifth', ' final'],
 '16': [' final', ' seventh', 'ighth', 'each', ' ninth'],
 '17': [' seventh', ' fifth', ' eighth', ' sixth', 'ighth'],
 '18': 

# Writing direction results with scatterplot

In [10]:
def scatter_attention_and_contribution(
    model,
    layer_no,
    head_no,
    dataset,
    S1_is_first=False,
    return_vals=False,
    return_fig=False,
):
    """
    Plot a scatter plot
    for each input sequence with the attention paid to S
    and the amount that is written in the S directions
    """

    n_heads = model.cfg.n_heads
    n_layers = model.cfg.n_layers
    model_unembed = model.unembed.W_U.detach().cpu()
    df = []
    cache = {}
    model.cache_all(cache)

    logits = model(dataset.toks.long())

    for i, prompt in enumerate(dataset.prompts):
        s_toks = []
        s_positions = []
        s_dirs = []

        targ_tokens = [key for key in dataset.prompts[0].keys() if key != 'text']
        for s_id in targ_tokens:
            if S1_is_first and s_id == "S1":  # only use this if first token doesn't have space Ġ in front
                s_tok = model.tokenizer(prompt["S1"])["input_ids"][0]
            else:
                s_tok = model.tokenizer(" " + prompt[s_id])["input_ids"][0]
            s_toks.append(s_tok)

            toks = model.tokenizer(prompt["text"])["input_ids"]
            try:
                s_pos = toks.index(s_tok)
            except ValueError:
                print(f"{s_tok} is not present in {toks}. Skipping...")
                continue

            s_pos = toks.index(s_tok)
            s_positions.append(s_pos)

            s_dir = model_unembed[:, s_tok].detach()
            s_dirs.append(s_dir)

        for dire, posses, tok_type in zip(s_dirs, s_positions, targ_tokens):
            prob = sum(
                [
                    cache[f"blocks.{layer_no}.attn.hook_attn"][
                        i, head_no, dataset.word_idx["end"][i], pos
                    ]
                    .detach()
                    .cpu()
                    for pos in [posses]
                ]
            )
            resid = (
                cache[f"blocks.{layer_no}.attn.hook_result"][
                    i, dataset.word_idx["end"][i], head_no, :
                ]
                .detach()
                .cpu()
            )
            dot = torch.einsum("a,a->", resid, dire)
            df.append([prob, dot, tok_type, prompt["text"]])

    viz_df = pd.DataFrame(
        df, columns=[f"Attn Prob on Number", f"Dot w Number Embed", "Seq Position", "text"]
    )
    fig = px.scatter(
        viz_df,
        x=f"Attn Prob on Number",
        y=f"Dot w Number Embed",
        color="Seq Position",
        hover_data=["text"],
        title=f"How Strong {layer_no}.{head_no} Writes in the Number Embed Direction Relative to Attn Prob",
    )

    if return_vals:
        return viz_df
    if return_fig:
        return fig
    else:
        fig.show()

In [11]:
scatter_attention_and_contribution(
    model=model, layer_no=9, head_no=1, dataset=dataset, S1_is_first=False
)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen

## Correlation vals

In [12]:
def get_prob_dot(  # same as scatterplot, but output x and y vals instead of plotting
    model,
    layer_no,
    head_no,
    dataset,
    S1_is_first=False,
    return_vals=False,
    return_fig=False,
):
    """
    Plot a scatter plot
    for each input sequence with the attention paid to S
    and the amount that is written in the S directions
    """

    n_heads = model.cfg.n_heads
    n_layers = model.cfg.n_layers
    model_unembed = model.unembed.W_U.detach().cpu()
    # df = []
    all_prob = []
    all_dot = []
    cache = {}
    model.cache_all(cache)

    logits = model(dataset.toks.long())

    for i, prompt in enumerate(dataset.prompts):
        s_toks = []
        s_positions = []
        s_dirs = []

        targ_tokens = [key for key in dataset.prompts[0].keys() if key != 'text']
        for s_id in targ_tokens:
            if S1_is_first and s_id == "S1":  # only use this if first token doesn't have space Ġ in front
                s_tok = model.tokenizer(prompt["S1"])["input_ids"][0]
            else:
                s_tok = model.tokenizer(" " + prompt[s_id])["input_ids"][0]
            s_toks.append(s_tok)

            toks = model.tokenizer(prompt["text"])["input_ids"]
            try:
                s_pos = toks.index(s_tok)
            except ValueError:
                print(f"{s_tok} is not present in {toks}. Skipping...")
                continue

            s_pos = toks.index(s_tok)
            s_positions.append(s_pos)

            s_dir = model_unembed[:, s_tok].detach()
            s_dirs.append(s_dir)

        for dire, posses, tok_type in zip(s_dirs, s_positions, targ_tokens):
            prob = sum(
                [
                    cache[f"blocks.{layer_no}.attn.hook_attn"][
                        i, head_no, dataset.word_idx["end"][i], pos
                    ]
                    .detach()
                    .cpu()
                    for pos in [posses]
                ]
            )
            resid = (
                cache[f"blocks.{layer_no}.attn.hook_result"][
                    i, dataset.word_idx["end"][i], head_no, :
                ]
                .detach()
                .cpu()
            )
            dot = torch.einsum("a,a->", resid, dire)
            #df.append([prob, dot, tok_type, prompt["text"]])
            all_prob.append(prob)
            all_dot.append(dot)

    return all_prob, all_dot


In [13]:
all_prob, all_dot = get_prob_dot(
    model=model, layer_no=9, head_no=9, dataset=dataset, S1_is_first=False
)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen

In [14]:
import scipy.stats as stats

# X and Y should be arrays, lists, or pandas Series
correlation, p_value = stats.pearsonr(all_prob, all_dot)

print("Correlation:", correlation)
print("p-value:", p_value)

Correlation: 0.8127540109290008
p-value: 8.99435128806603e-70


In [15]:
all_prob, all_dot = get_prob_dot(
    model=model, layer_no=9, head_no=1, dataset=dataset, S1_is_first=False
)

correlation, p_value = stats.pearsonr(all_prob, all_dot)
print("Correlation:", correlation)
print("p-value:", p_value)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen

In [16]:
all_prob, all_dot = get_prob_dot(
    model=model, layer_no=7, head_no=10, dataset=dataset, S1_is_first=False
)

correlation, p_value = stats.pearsonr(all_prob, all_dot)
print("Correlation:", correlation)
print("p-value:", p_value)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen

In [17]:
all_prob, all_dot = get_prob_dot(
    model=model, layer_no=5, head_no=1, dataset=dataset, S1_is_first=False
)

correlation, p_value = stats.pearsonr(all_prob, all_dot)
print("Correlation:", correlation)
print("p-value:", p_value)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen

In [18]:
all_prob, all_dot = get_prob_dot(
    model=model, layer_no=0, head_no=3, dataset=dataset, S1_is_first=False
)

correlation, p_value = stats.pearsonr(all_prob, all_dot)
print("Correlation:", correlation)
print("p-value:", p_value)

352 is not present in [16, 362, 513, 604]. Skipping...
362 is not present in [17, 513, 604, 642]. Skipping...
513 is not present in [18, 604, 642, 718]. Skipping...
604 is not present in [19, 642, 718, 767]. Skipping...
642 is not present in [20, 718, 767, 807]. Skipping...
718 is not present in [21, 767, 807, 860]. Skipping...
767 is not present in [22, 807, 860, 838]. Skipping...
807 is not present in [23, 860, 838, 1367]. Skipping...
860 is not present in [24, 838, 1367, 1105]. Skipping...
838 is not present in [940, 1367, 1105, 1511]. Skipping...
1367 is not present in [1157, 1105, 1511, 1478]. Skipping...
1105 is not present in [1065, 1511, 1478, 1315]. Skipping...
1511 is not present in [1485, 1478, 1315, 1467]. Skipping...
1478 is not present in [1415, 1315, 1467, 1596]. Skipping...
1315 is not present in [1314, 1467, 1596, 1248]. Skipping...
1467 is not present in [1433, 1596, 1248, 678]. Skipping...
1596 is not present in [1558, 1248, 678, 1160]. Skipping...
1248 is not presen